# **Bitcoin price prediction - Linear Regression**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: executing the chosen model, first with default values, then by choosing the best parameters by performing hyperparameter tuning with cross validation and performance evaluation.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = True # Decide whether or not to use operations that might slow down notebook execution
SPLITTING_METHOD = "single_split"
MODEL_NAME = "LinearRegression"
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

Mounted at /content/drive


In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets names
DATASET_TRAIN_VALID_NAME = "bitcoin_blockchain_data_15min_train_valid"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features labels
FEATURES_LABEL = "features"
TARGET_LABEL = "next-market-price"

# Features names
ALL_FEATURES_NAME = "all_features"
MOST_REL_FEATURES_NAME = "most_rel_features"
LEAST_REL_FEATURES_NAME = "least_rel_features"

# Features paths
ALL_FEATURES = FEATURES_DIR + "/" + ALL_FEATURES_NAME + ".json"
MOST_REL_FEATURES = FEATURES_DIR + "/" + MOST_REL_FEATURES_NAME + ".json"
LEAST_REL_FEATURES = FEATURES_DIR + "/" + LEAST_REL_FEATURES_NAME + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

#####################
# --- UTILITIES --- #
#####################

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + ".csv"
MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [4]:
# Suppression of warnings for better reading
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# For cleaning temp outputs
from IPython.display import clear_output

In [5]:
if not LOCAL_RUNNING:
    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=54332b9025f8c5726a33fd64ca6c7c1ee97b5a46fc51f69900004a72515a5739
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

# Import files

In [6]:
# Import my files
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import utilities, parameters

importlib.reload(utilities)
importlib.reload(parameters)

<module 'parameters' from '/content/drive/MyDrive/BDC/project/utilities/parameters.py'>

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
def dataset_info(dataset):
  # Print dataset
  dataset.show(3)

  # Get the number of rows
  num_rows = dataset.count()

  # Get the number of columns
  num_columns = len(dataset.columns)

  # Print the shape of the dataset
  print("Shape:", (num_rows, num_columns))

  # Print the schema of the dataset
  dataset.printSchema()

In [10]:
if SLOW_OPERATIONS:
  dataset_info(df)

+-------------------+---+------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+--------------------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|      total-bitcoins|          market-cap|        trade-volume|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|   n-transactions|estimated-transaction-volume-usd|      rate-of-change|        sma-5-days|        sma-7-days|       sma-10-days|       sma-20-days|       sma-50-days|      sma-100-days| next-market-price|
+-----

Instead of considering the entire dataset, I will only consider the last two years from today's date in order to train / validate the model.

In [11]:
# Retrieve the last timestamp value
last_value = df.agg(last("timestamp")).collect()[0][0]

# Consider only the last two years as of today
split_date = last_value - relativedelta(years=2)

# Split the dataset based on the desired date and drop the "id" column
df = df[df['timestamp'] > split_date].drop(col("id"))

# Recompute id column
df = df.withColumn("id", F.row_number().over(Window.orderBy(F.monotonically_increasing_id()))-1)
# Rearranges columns
new_columns = ["timestamp", "id"] + [col for col in df.columns if col not in ["timestamp", "id", "next-market-price"]] + ["next-market-price"]
df = df.select(*new_columns)

In [12]:
if SLOW_OPERATIONS:
  dataset_info(df)

+-------------------+---+------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+--------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|      total-bitcoins|          market-cap|        trade-volume|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|           hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|      rate-of-change|       sma-5-days|        sma-7-days|       sma-10-days|       sma-20-days|       sma-50-days|      sma-100-days| next-market-price|
+-----------

# Loading features

In [13]:
# Loading all the features
with open(ALL_FEATURES, "r") as f:
    ALL_FEATURES = json.load(f)
print(ALL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume', 'sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


In [14]:
# Loading the most relevant features
with open(MOST_REL_FEATURES, "r") as f:
    MOST_REL_FEATURES = json.load(f)
print(MOST_REL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume']


In [15]:
# Loading least relevant features
with open(LEAST_REL_FEATURES, "r") as f:
    LEAST_REL_FEATURES = json.load(f)
print(LEAST_REL_FEATURES)

['sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- **Default without normalization:** Make predictions using the chosen base model
- **Default with normalization:** Like the previous one but features are normalized

At this point, the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:

- **Hyperparameter tuning:** Researching the best parameters to use
- **Cross Validation:** Validate the performance of the model with the chosen parameters

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved to later make predictions on the test set.

---

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem):

- **Block time series splits:** Involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

    ![block-splits.png](images/block-splits.png)

- **Walk forward time series splits:** Involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

    ![walk-forward-splits.png](images/walk-forward-splits.png)

- **Single time series split:** Involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

    ![single-split.png](images/single-split.png)

In [16]:
# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'single_split', 'split_label': 'months', 'split_value': 1}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

### Without normalization

In [17]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [18]:
# Get default parameters
params = parameters.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [100], 'regParam': [0.0], 'elasticNetParam': [0.0]}

In [19]:
# Make predictions by using all the features
default_res_all, default_pred_all = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_res_all

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,single_split,all_features,"[100, 0.0, 0.0]",3.302319,10.905312,2.795275,0.000105,0.999973,0.999973,6.076524


In [20]:
utilities.show_results(default_pred_all, MODEL_NAME)

In [21]:
# Make predictions by using the most relevant features
default_res_most_rel, default_pred_most_rel = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_res_most_rel

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,single_split,most_rel_features,"[100, 0.0, 0.0]",6.209474,38.557567,4.733744,0.000177,0.999906,0.999906,2.696436


In [22]:
utilities.show_results(default_pred_most_rel, MODEL_NAME)

In [23]:
# Make predictions by using the least relevant features
default_res_least_rel, default_pred_least_rel = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_res_least_rel

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,single_split,least_rel_features,"[100, 0.0, 0.0]",2993.399802,8.960442e+06,2536.922377,0.095796,-20.816434,-20.845806,3.201399


In [24]:
utilities.show_results(default_pred_least_rel, MODEL_NAME)

### With normalization

In [25]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [26]:
# Valid performances with all the features
default_norm_res_all, default_norm_pred_all = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_norm_res_all

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,single_split,all_features,"[100, 0.0, 0.0]",3160.660276,9.989773e+06,3084.616396,0.115801,-23.322597,-23.355344,2.401957


In [27]:
utilities.show_results(default_norm_pred_all, MODEL_NAME)

In [28]:
# Make predictions by using the most relevant features
default_norm_res_most_rel, default_norm_pred_most_rel = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_norm_res_most_rel

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,single_split,most_rel_features,"[100, 0.0, 0.0]",13821.934212,1.910459e+08,13626.106854,0.511417,-464.148846,-464.775098,2.551458


In [29]:
utilities.show_results(default_norm_pred_most_rel, MODEL_NAME)

In [30]:
# Make predictions by using the least relevant features
default_norm_res_least_rel, default_norm_pred_least_rel = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
default_norm_res_least_rel

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,single_split,least_rel_features,"[100, 0.0, 0.0]",15285.715572,2.336531e+08,14810.715227,0.557392,-567.886795,-568.652715,1.844674


In [31]:
utilities.show_results(default_norm_pred_least_rel, MODEL_NAME)

In [32]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [33]:
# Concatenate default results into Pandas dataset
default_comparison_lst_df = pd.DataFrame(pd.concat([default_res_all, default_res_most_rel, default_res_least_rel, default_norm_res_all, default_norm_res_most_rel, default_norm_res_least_rel], ignore_index=True))
default_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,single_split,all_features,"[100, 0.0, 0.0]",3.302319,1.090531e+01,2.795275,0.000105,0.999973,0.999973,6.076524
1,LinearRegression,default,single_split,most_rel_features,"[100, 0.0, 0.0]",6.209474,3.855757e+01,4.733744,0.000177,0.999906,0.999906,2.696436
2,LinearRegression,default,single_split,least_rel_features,"[100, 0.0, 0.0]",2993.399802,8.960442e+06,2536.922377,0.095796,-20.816434,-20.845806,3.201399
3,LinearRegression,default_norm,single_split,all_features,"[100, 0.0, 0.0]",3160.660276,9.989773e+06,3084.616396,0.115801,-23.322597,-23.355344,2.401957
4,LinearRegression,default_norm,single_split,most_rel_features,"[100, 0.0, 0.0]",13821.934212,1.910459e+08,13626.106854,0.511417,-464.148846,-464.775098,2.551458
5,LinearRegression,default_norm,single_split,least_rel_features,"[100, 0.0, 0.0]",15285.715572,2.336531e+08,14810.715227,0.557392,-567.886795,-568.652715,1.844674


## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration.

In [ ]:
# Using one of the multiple splitting methods to tune the model
SPLITTING_METHOD = "walk_forward_splits"

# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

### Hyperparameter tuning

In [ ]:
# From now on, only selected and normalized features will be considered
MODEL_TYPE = "hyp_tuning"
CHOSEN_FEATURES = MOST_REL_FEATURES
CHOSEN_FEATURES_LABEL = MOST_REL_FEATURES_NAME
FEATURES_NORMALIZATION = True

In [ ]:
# Get model grid parameters
params = parameters.get_model_grid_params(MODEL_NAME)
params

In [ ]:
# Perform hyperparameter tuning
hyp_res = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
hyp_res

---
To select the best parameters to be used in the final model I assign a score to each value in the "Parameters" column based on the following criteria:
* Calculate the frequency of each unique value in the "Parameters" column.
* Normalize the frequencies to a scale of 0 to 1, where 1 represents the highest frequency.
* Calculate the split weight for each value in the "Parameters" column, where a higher split number corresponds to a higher weight.
* Normalize the split weights to a scale of 0 to 1, where 1 represents the highest split weight.
* Calculate the RMSE weight for each value in the "Parameters" column, where a lower RMSE value corresponds to a higher weight.
* Normalize the RMSE weights to a scale of 0 to 1, where 1 represents the highest RMSE weight.

Then calculate the overall score for each value in the "Parameters" column by combining the normalized frequency, split weight, and RMSE weight and take into consideration the parameters that have the highest score.

In [ ]:
# Show parameters score
grouped_scores, best_params = parameters.choose_best_params(hyp_res)
grouped_scores

In [ ]:
# Print best parameters
print(f"Best parameters: {best_params}")

### Cross validation

In [ ]:
MODEL_TYPE = "cross_val"

In [ ]:
# Get tuned parameters
params = parameters.get_best_model_params(best_params, MODEL_NAME)
params

In [ ]:
# Perform cross validation
cv_res, cv_pred = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
cv_res

In [ ]:
utilities.show_results(cv_pred, MODEL_NAME)

In [ ]:
# Define the results to show
tuned_comparison_lst = [cv_res]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

## Validated
Let's see if the situation has improved by validating the model with the newly found parameters.

In [ ]:
# Using single split method to validate the model
SPLITTING_METHOD = "single_split"

# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

In [ ]:
MODEL_TYPE = "final_validated"

In [ ]:
# Validate the model with the best parameters
valid_res, valid_pred = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
valid_res

In [ ]:
utilities.show_results(valid_pred, MODEL_NAME)

## Trained
At this point it is possible to train the final model over the entire train / validation set in order to use it in the final step, that is, to make predictions about the data contained in the test set.

In [ ]:
MODEL_TYPE = "final_trained"

In [ ]:
# Train the final model on the whole train / validation set
train_res, final_model, train_pred = utilities.evaluate_trained_model(df, params, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
clear_output(wait=True)
train_res

In [ ]:
utilities.show_results(train_pred, MODEL_NAME)

# Comparison table
Visualization of model performance at various stages of train / validation

In [ ]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df, valid_res, train_res], ignore_index=True))
final_comparison_lst_df

# Model accuracy

Since predicting the price accurately is very difficult let's se how good the model is at predicting whether the price will go up or down.

For each row in our predictions let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
We compute whether the current prediction is correct (1) or not (0):

$$
prediction\_is\_correct
=
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that we count the number of correct prediction:
$$
correct\_predictions
=
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally we compute the percentage of accuracy of the model:
$$
\\
accuracy
=
(correct\_predictions / total\_rows)
* 100
$$

In [ ]:
# Convert the pandas dataset to a PySpark dataset
final_valid_pred_spark = spark.createDataFrame(valid_pred)

# Compute model accuracy
accuracy = utilities.model_accuracy(final_valid_pred_spark)

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy': accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD}: {accuracy:.2f}%")

# Saving final results


In [ ]:
# Saving final model results
final_comparison_lst_df.to_csv(MODEL_RESULTS, index=False)

In [ ]:
# Saving final accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [ ]:
# Saving final model
final_model.write().overwrite().save(MAIN_DIR + '/models/LinearRegression')